In [1]:
import os
import json
from pprint import pprint

# List to hold the data from all JSON files
all_json_data = {}

# Get the current directory
current_directory = os.getcwd()

# Loop through all files in the current directory
for filename in os.listdir(current_directory):
    if filename.endswith('.json'):
        file_path = os.path.join(current_directory, filename)
        
        # Open and read the JSON file
        with open(file_path, 'r') as file:
            try:
                json_data = json.load(file)

                label = filename.split('.')[0]

                all_json_data[label] = json_data
            except json.JSONDecodeError as e:
                print(f"Error reading {filename}: {e}")

# Now, all_json_data contains the contents of all JSON files
pprint(all_json_data.keys())


dict_keys(['openai_gmm_failure_cases_image_few_shot', '', 'openai_rabi_success_cases_image_few_shot', 'llama_rabi_failure_cases_image_few_shot', 'gemini_rabi_success_cases_raw', 'llama_drag_success_cases_raw', 'openai_drag_failure_cases_image_few_shot', 'llama_gmm_success_cases_raw', 'gemini_resonator_spec_failure_cases_raw', 'llama_rabi_success_cases_image_few_shot', 'openai_gmm_success_cases_image_few_shot', 'gemini_drag_failure_cases_raw', 'llama_rabi_failure_cases_raw', 'llama_drag_failure_cases_raw', 'openai_drag_success_cases_raw', 'gemini_drag_failure_cases_image_few_shot', 'llama_gmm_failure_cases_image_few_shot', 'gemini_gmm_success_cases_image_few_shot', 'openai_rabi_failure_cases_raw', 'gemini_rabi_failure_cases_raw', 'gemini_resonator_spec_success_cases_raw', 'llama_gmm_failure_cases_raw', 'gemini_resonator_spec_success_cases_image_few_shot', 'openai_drag_success_cases_image_few_shot', 'openai_drag_failure_cases_raw', 'openai_rabi_success_cases_raw', 'llama_rabi_success_cas

In [2]:
import numpy as np
import pandas as pd

In [3]:
entries = []

def summarize_statistics(data):

    entries = []
    for key,val in data.items():
        data = {
            'model_type':key
        }
        data.update(val)
        entries.append(data)

    return entries

for case,data in all_json_data.items():
    model = case.split('_')[0]
    case = '_'.join(case.split('_')[1:-1])
    for _, results in data.items():
        entry = {}
        entry['case'] = case
        entry['model'] = model
        for r in summarize_statistics(results):
                r.update(entry)
                entries.append(r)

In [7]:
print(entries[0].keys())

dict_keys(['model_type', 'full', 'fitting_only', 'visual_only', 'case', 'model'])


In [4]:
df = pd.DataFrame(entries)

In [5]:
df['Experiment success'] = df['Experiment success']==True

In [6]:
mask = df['case'].str.contains('success')

In [7]:
df['success'] = df['Experiment success']==mask

In [8]:
sub_df= df[['model_type','Experiment success','case', 'model']]
mask_sub_df = sub_df['model'] == 'openai'
sub_df[mask_sub_df].groupby(['case','model_type', 'model']).mean()

Experiment success
case         model_type   model                     
rabi_failure fitting_only openai                1.00
             full         openai                0.05
             visual_only  openai                0.05
rabi_success fitting_only openai                1.00
             full         openai                1.00
             visual_only  openai                1.00

In [9]:
sub_df= df[['model_type','success','case', 'model']]
mask_sub_df = sub_df['model'] == 'openai'
sub_df[mask_sub_df].groupby(['case','model_type', 'model']).mean()

success
case         model_type   model          
rabi_failure fitting_only openai     0.00
             full         openai     0.95
             visual_only  openai     0.95
rabi_success fitting_only openai     1.00
             full         openai     1.00
             visual_only  openai     1.00

In [10]:
sub_df= df[['model_type','success']]
# mask_sub_df = sub_df['model'] == 'open'
sub_df[mask_sub_df].groupby(['model_type']).mean()

,success
model_type,
fitting_only,0.500
full,0.975
visual_only,0.975
